### A notebook for geocoding

In [1]:
import requests
import time
from IPython.display import clear_output

#### A function for geocoding our locations


In [2]:
def getLocation(query):
    url = 'https://maps.googleapis.com/maps/api/geocode/json'
    params = {'sensor': 'false', 'address': '{}'.format(query)}
    r = requests.get(url, params=params)
    results = r.json()['results']
    location = results[0]['geometry']['location']
    return location['lat'], location['lng']
    

#### Example of getting a location

In [3]:
getLocation("Burkina Faso,Ouagadougou (urban)")

(12.3714277, -1.5196603)

#### Read in the unified data from R

In [1]:
places = open('uniquePlaces.csv', 'r').readlines()
places = places[1:]


In [21]:
print(places[0:5])

['"1","Angola, Luanda"\n', '"2","Benin, Cotonou"\n', '"3","Benin, Atacora District"\n', '"4","Botswana, Gaberone"\n', '"5","Burkina Faso, Bobo Dioulasso"\n']


#### A loop to query each of our hospitals or clinics

In [25]:
outFile = open("placesWithlocs.csv", "w")
n = 0
k = len(places)
retries = 30
for place in places:
    place = str(place).split(",", 1)[1]
    outFile = open("placesWithlocs.csv", "a")
    tmp = str(place).replace("\n", "")
    print(tmp)
    
    for i in range(retries):
        time.sleep(0.5)
        try:
            loc  = getLocation(tmp)
            print("loc found")
            n += 1
             
        except IndexError:
            print("loc failed")
            print("retry: {}/{}".format(i, retries))   
            continue
        else:
            break
    else:
        loc = "'NULL', 'NULL'"
        n += 1       
        
    
    clear_output(wait=True)
    print(loc)
    print ("Progress: {}/{}".format(n,k))   
    
    result = str(loc[0]) + "|" + str(loc[1]) + "|" + place
    
    print(result)
    
    outFile.write(result)
    outFile.close()
    

(-17.8251657, 31.03351)
Progress: 76/76
-17.8251657|31.03351|"Zimbabwe, Harare"



#### Explore the sites which failed geocoding 

In [12]:
! cat placesWithlocs.csv | wc -l 

0


In [ ]:
!cat placesWithlocs.csv | grep NULL | wc -l

In [ ]:
!cat placesWithlocs.csv